In [72]:
%load_ext autoreload 
%autoreload 2
from lens_simulation import SimulationRunner, Lens


sim_runner = SimulationRunner.SimulationRunner()
sim_runner.initialise_simulation()
sim_runner.parameters = { "lens ": [
            Lens.Lens(1, 2, 2.0, Lens.Medium(2.348)), 
            Lens.Lens(1, 2, 2.1, Lens.Medium(2.348)),
            Lens.Lens(1, 2, 2.2, Lens.Medium(2.348))
        ],
                            "medium": [3, 4], 
                            "another": [5, 6]
        }
sim_runner.setup_simulation()
sim_runner.run_simulations()

# Test github

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Hello Sim:  1bc9c1da-2686-4d96-840c-125578fb1d85
Running 12 Simulation
--------------------------------------------------
Running Simulation main-hyena (32a2ceac4a)
Parameters:  {'lens ':  Lens (diameter: 1, height: 2, medium: 2.0, 'medium': 3, 'another': 5}
--------------------------------------------------
--------------------------------------------------
Running Simulation normal-mullet (98f575fe58)
Parameters:  {'lens ':  Lens (diameter: 1, height: 2, medium: 2.0, 'medium': 3, 'another': 6}
--------------------------------------------------
--------------------------------------------------
Running Simulation nearby-sawfly (00a0328553)
Parameters:  {'lens ':  Lens (diameter: 1, height: 2, medium: 2.0, 'medium': 4, 'another': 5}
--------------------------------------------------
--------------------------------------------------
Running Simulation sharp-guppy (765d1f5399)
Parameters:  {'lens ': 

'big-hyena'